# Project 1: Trading with Momentum
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.

### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 880kB 9.4MB/s eta 0:00:01
    100% |████████████████████████████████| 17.0MB 1.1MB/s eta 0:00:01  1% |▋                               | 337kB 29.2MB/s eta 0:00:01    16% |█████▎                          | 2.8MB 12.9MB/s eta 0:00:02    20% |██████▌                         | 3.4MB 13.7MB/s eta 0:00:01    23% |███████▋                        | 4.0MB 13.9MB/s eta 0:00:01    30% |█████████▉                      | 5.2MB 9.9MB/s eta 0:00:02    72% |███████████████████████▍        | 12.4MB 25.1MB/s eta 0:00:01    82% |██████████████████████████▍     | 14.0MB 10.3MB/s eta 0:00:01    95% |██████████████████████████████▌ | 16.2MB 26.3MB/s eta 0:00:01    98% |███████████████████████████████▋| 16.8MB 13.1MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 834kB/s ta 0:00:011  1% |▋                               | 491kB 11.5MB/s eta 0:00:03    6% |██                              | 1.6MB 11.5MB/s eta 0:00:03    8% |██▋                       

    100% |████████████████████████████████| 50.0MB 369kB/s ta 0:00:011 1% |▌                               | 727kB 6.8MB/s eta 0:00:08    2% |▊                               | 1.1MB 9.3MB/s eta 0:00:06    5% |█▊                              | 2.8MB 10.0MB/s eta 0:00:05    7% |██▎                             | 3.6MB 8.9MB/s eta 0:00:06    8% |██▋                             | 4.0MB 8.2MB/s eta 0:00:06    9% |███                             | 4.8MB 8.9MB/s eta 0:00:06    12% |████                            | 6.1MB 10.2MB/s eta 0:00:05    15% |█████                           | 7.8MB 9.3MB/s eta 0:00:05    16% |█████▎                          | 8.2MB 7.7MB/s eta 0:00:06    18% |█████▉                          | 9.1MB 8.7MB/s eta 0:00:05    19% |██████▍                         | 10.0MB 9.4MB/s eta 0:00:05    20% |██████▋                         | 10.4MB 7.5MB/s eta 0:00:06    21% |██████▉                         | 10.7MB 9.1MB/s eta 0:00:05    24% |███████▊                        | 12.0MB 

  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /root/.cache/pip/wheels/98/54/81/dd92d5b0858fac680cd7bdb8800eb26c001dd9f5dc8b1bc0ba
  Running setup.py bdist_wheel for ecos ... done
  Stored in directory: /root/.cache/pip/wheels/34/83/48/2192df21a678bf0fde91e59ec6ff78525bb3d99919397d6a8c
  Running setup.py bdist_wheel for scs ... done
  Stored in directory: /root/.cache/pip/wheels/96/db/82/bdbce7e4bc907672294aac2cc582b71fbf908b2fb1bd65156c
  Running setup.py bdist_wheel for qdldl ... done
  Stored in directory: /root/.cache/pip/wheels/a9/77/d6/726fc4a2ae1513b4663b81721f5d75e9b4fe9d74ca7a8a5417
Successfully built cvxpy plotly ecos scs qdldl
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
moviepy 0.2.3.2 has requirement tqdm==4.11.2, but you'll have tqdm 4.19.5 which is incompatible.
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
      Successfully uninstalled numpy-1.12.1
  Found existing instal

### Load Packages

In [2]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
The data we use for most of the projects is end of day data. This contains data for many stocks, but we'll be looking at stocks in the S&P 500. We also made things a little easier to run by narrowing down our range of time period instead of using all of the data.

In [3]:
df = pd.read_csv('../../data/project_1/eod-quotemedia.csv', parse_dates=['date'], index_col=False)

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

print('Loaded Data')

Loaded Data


### View Data
Run the cell below to see what the data looks like for `close`.

In [4]:
project_helper.print_dataframe(close)

### Stock Example
Let's see what a single stock looks like from the closing prices. For this example and future display examples in this project, we'll use Apple's stock (AAPL). If we tried to graph all the stocks, it would be too much information.

In [5]:
apple_ticker = 'AAPL'
project_helper.plot_stock(close[apple_ticker], '{} Stock'.format(apple_ticker))

## Resample Adjusted Prices

The trading signal you'll develop in this project does not need to be based on daily prices, for instance, you can use month-end prices to perform trading once a month. To do this, you must first resample the daily adjusted closing prices into monthly buckets, and select the last observation of each month.

Implement the `resample_prices` to resample `close_prices` at the sampling frequency of `freq`.

In [6]:
def resample_prices(close_prices, freq='M'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    # TODO: Implement Function
    print(freq)
    
    return close_prices.resample(freq).last()

project_tests.test_resample_prices(resample_prices)

M
Tests Passed


### View Data
Let's apply this function to `close` and view the results.

In [7]:
monthly_close = resample_prices(close)
project_helper.plot_resampled_prices(
    monthly_close.loc[:, apple_ticker],
    close.loc[:, apple_ticker],
    '{} Stock - Close Vs Monthly Close'.format(apple_ticker))

M


## Compute Log Returns

Compute log returns ($R_t$) from prices ($P_t$) as your primary momentum indicator:

$$R_t = log_e(P_t) - log_e(P_{t-1})$$

Implement the `compute_log_returns` function below, such that it accepts a dataframe (like one returned by `resample_prices`), and produces a similar dataframe of log returns. Use Numpy's [log function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html) to help you calculate the log returns.

In [8]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    # TODO: Implement Function
    logclose = np.log(prices)
    logrtn = logclose - logclose.shift(1)
    
    return logrtn

project_tests.test_compute_log_returns(compute_log_returns)

Tests Passed


### View Data
Using the same data returned from `resample_prices`, we'll generate the log returns.

In [9]:
monthly_close_returns = compute_log_returns(monthly_close)
project_helper.plot_returns(
    monthly_close_returns.loc[:, apple_ticker],
    'Log Returns of {} Stock (Monthly)'.format(apple_ticker))

## Shift Returns
Implement the `shift_returns` function to shift the log returns to the previous or future returns in the time series. For example, the parameter `shift_n` is 2 and `returns` is the following:

```
                           Returns
               A         B         C         D
2013-07-08     0.015     0.082     0.096     0.020     ...
2013-07-09     0.037     0.095     0.027     0.063     ...
2013-07-10     0.094     0.001     0.093     0.019     ...
2013-07-11     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...
```

the output of the `shift_returns` function would be:
```
                        Shift Returns
               A         B         C         D
2013-07-08     NaN       NaN       NaN       NaN       ...
2013-07-09     NaN       NaN       NaN       NaN       ...
2013-07-10     0.015     0.082     0.096     0.020     ...
2013-07-11     0.037     0.095     0.027     0.063     ...
...            ...       ...       ...       ...
```
Using the same `returns` data as above, the `shift_returns` function should generate the following with `shift_n` as -2:
```
                        Shift Returns
               A         B         C         D
2013-07-08     0.094     0.001     0.093     0.019     ...
2013-07-09     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...       ...
...            ...       ...       ...       ...       ...
...            NaN       NaN       NaN       NaN       ...
...            NaN       NaN       NaN       NaN       ...
```
_Note: The "..." represents data points we're not showing._

In [10]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """
    # TODO: Implement Function
    
    return returns.shift(shift_n)

project_tests.test_shift_returns(shift_returns)

Tests Passed


### View Data
Let's get the previous month's and next month's returns.

In [11]:
prev_returns = shift_returns(monthly_close_returns, 1)
lookahead_returns = shift_returns(monthly_close_returns, -1)

project_helper.plot_shifted_returns(
    prev_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Previous Returns of {} Stock'.format(apple_ticker))
project_helper.plot_shifted_returns(
    lookahead_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Lookahead Returns of {} Stock'.format(apple_ticker))

## Generate Trading Signal

A trading signal is a sequence of trading actions, or results that can be used to take trading actions. A common form is to produce a "long" and "short" portfolio of stocks on each date (e.g. end of each month, or whatever frequency you desire to trade at). This signal can be interpreted as rebalancing your portfolio on each of those dates, entering long ("buy") and short ("sell") positions as indicated.

Here's a strategy that we will try:
> For each month-end observation period, rank the stocks by _previous_ returns, from the highest to the lowest. Select the top performing stocks for the long portfolio, and the bottom performing stocks for the short portfolio.

Implement the `get_top_n` function to get the top performing stock for each month. Get the top performing stocks from `prev_returns` by assigning them a value of 1. For all other stocks, give them a value of 0. For example, using the following `prev_returns`:

```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0.015     0.082     0.096     0.020     0.075     0.043     0.074
2013-07-09     0.037     0.095     0.027     0.063     0.024     0.086     0.025
...            ...       ...       ...       ...       ...       ...       ...
```

The function `get_top_n` with `top_n` set to 3 should return the following:
```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0         1         1         0         1         0         0
2013-07-09     0         1         0         1         0         1         0
...            ...       ...       ...       ...       ...       ...       ...
```
*Note: You may have to use Panda's [`DataFrame.iterrows`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.iterrows.html) with [`Series.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nlargest.html) in order to implement the function. This is one of those cases where creating a vecorization solution is too difficult.*

In [12]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    # TODO: Implement Function
    new_df = pd.DataFrame(0, index=prev_returns.index, columns=prev_returns.columns)
#    print(new_df.loc['2008-10-31'][new_df.columns[2]])
#    print(prev_returns.loc['2008-10-31',columns[2]])
#    print(prev_returns.loc['2008-10-31'])
    
    print(prev_returns)
    for index, row in prev_returns.iterrows():
        print('-----1')
        print(index)
        print('-----2')
        print(row)
        print('-----3')
        toptick = row.nlargest(top_n).index
        new_df.loc[index][toptick] = 1

    print(new_df)
    
    
    return new_df

project_tests.test_get_top_n(get_top_n)

                   CTW         RYL         IZL        AWJV         OOO
2008-08-31         nan         nan         nan         nan         nan
2008-09-30         nan         nan         nan         nan         nan
2008-10-31  3.13172138  0.72709204  5.76874778  1.77557845  0.04098317
2008-11-30 -3.78816218 -0.67583590 -4.95433863 -1.67093250 -0.24929051
-----1
2008-08-31 00:00:00
-----2
CTW    nan
RYL    nan
IZL    nan
AWJV   nan
OOO    nan
Name: 2008-08-31 00:00:00, dtype: float64
-----3
-----1
2008-09-30 00:00:00
-----2
CTW    nan
RYL    nan
IZL    nan
AWJV   nan
OOO    nan
Name: 2008-09-30 00:00:00, dtype: float64
-----3
-----1
2008-10-31 00:00:00
-----2
CTW    3.13172138
RYL    0.72709204
IZL    5.76874778
AWJV   1.77557845
OOO    0.04098317
Name: 2008-10-31 00:00:00, dtype: float64
-----3
-----1
2008-11-30 00:00:00
-----2
CTW    -3.78816218
RYL    -0.67583590
IZL    -4.95433863
AWJV   -1.67093250
OOO    -0.24929051
Name: 2008-11-30 00:00:00, dtype: float64
-----3
            CTW  R

### View Data
We want to get the best performing and worst performing stocks. To get the best performing stocks, we'll use the `get_top_n` function. To get the worst performing stocks, we'll also use the `get_top_n` function. However, we pass in `-1*prev_returns` instead of just `prev_returns`. Multiplying by negative one will flip all the positive returns to negative and negative returns to positive. Thus, it will return the worst performing stocks.

In [13]:
top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)
project_helper.print_top(df_long, 'Longed Stocks')
project_helper.print_top(df_short, 'Shorted Stocks')

ticker               A         AAL         AAP        AAPL        ABBV  \
date                                                                     
2013-07-31         nan         nan         nan         nan         nan   
2013-08-31         nan         nan         nan         nan         nan   
2013-09-30  0.04181412 -0.18015337 -0.02977582  0.08044762 -0.06518370   
2013-10-31  0.09657861  0.15979244  0.03282284 -0.02171531  0.04855545   
2013-11-30 -0.00960698  0.14734639  0.18195865  0.09201927  0.08860637   
2013-12-31  0.05388057  0.06647111  0.01828314  0.06772063  0.00000000   
2014-01-31  0.06769609  0.07267716  0.09197258  0.00886237  0.08616823   
2014-02-28  0.01664682  0.28421071  0.03663543 -0.11394918 -0.06220213   
2014-03-31 -0.02120344  0.09598737  0.10373913  0.05588347  0.03355617   
2014-04-30 -0.01790035 -0.00897599 -0.00629368  0.01975642  0.00957880   
2014-05-31 -0.03182502 -0.04270218 -0.04205794  0.09476126  0.02214224   
2014-06-30  0.05227357  0.13552541  0.

-----1
2014-06-30 00:00:00
-----2
ticker
A       0.05227357
AAL     0.13552541
AAP     0.02346722
AAPL    0.07577509
ABBV    0.04229556
ABC     0.11924492
ABT     0.03225676
ACN     0.01521647
ADBE    0.04516334
ADI     0.02805861
ADM     0.03269567
ADP     0.02182274
ADS     0.05684832
ADSK    0.08671632
AEE    -0.04860863
AEP     0.00071705
AES    -0.02451962
AET     0.08192499
AFL    -0.01810742
AGN     0.03467777
AIG     0.01753776
AIV     0.02923243
AIZ     0.00996580
AJG     0.01783206
AKAM    0.02364881
ALB     0.03156689
ALGN    0.08042427
ALK     0.04810892
ALL     0.02754317
ALLE    0.05977797
           ...    
VRTX    0.06518455
VTR     0.01083695
VZ      0.06683633
WAT     0.01630635
WBA     0.05737930
WDC    -0.00261468
WEC    -0.05443654
WFC     0.02976708
WHR    -0.06105928
WLTW           nan
WM      0.00516101
WMB     0.10758711
WMT    -0.03144298
WRK            nan
WU      0.01872714
WY      0.05846585
WYN     0.04045244
WYNN    0.05896748
XEC     0.08193807
XEL    -0

-----1
2015-08-31 00:00:00
-----2
ticker
A       0.05961780
AAL     0.00412320
AAP     0.08953503
AAPL   -0.03344115
ABBV    0.04841767
ABC    -0.00556369
ABT     0.03709245
ACN     0.06335602
ADBE    0.01202468
ADI    -0.09565300
ADM    -0.01672979
ADP    -0.00575002
ADS    -0.05963176
ADSK    0.01003436
AEE     0.08639194
AEP     0.06575309
AES    -0.02745748
AET    -0.11852280
AFL     0.02930903
AGN     0.08732164
AIG     0.03652939
AIV     0.05658660
AIZ     0.10744789
AJG     0.00274464
AKAM    0.09411158
ALB    -0.01954936
ALGN   -0.00015948
ALK     0.16185908
ALL     0.06099634
ALLE    0.04994553
           ...    
VRTX    0.08919558
VTR     0.07745006
VZ      0.01559287
WAT     0.03903195
WBA     0.13484803
WDC     0.09296573
WEC     0.08582470
WFC     0.02857087
WHR     0.02668497
WLTW           nan
WM      0.09815012
WMB    -0.08943793
WMT     0.01469483
WRK     0.03485345
WU     -0.00443678
WY     -0.02605068
WYN     0.00741960
WYNN    0.04517856
XEC    -0.05775050
XEL     0

Name: 2016-07-31 00:00:00, Length: 495, dtype: float64
-----3
-----1
2016-08-31 00:00:00
-----2
ticker
A       0.08115189
AAL     0.22631760
AAP     0.04966480
AAPL    0.08623527
ABBV    0.07621369
ABC     0.07139375
ABT     0.13578830
ACN    -0.00424592
ADBE    0.02137959
ADI     0.11950785
ADM     0.04979999
ADP    -0.03230012
ADS     0.16739169
ADSK    0.09356234
AEE    -0.02150621
AEP    -0.01133522
AES    -0.00153178
AET    -0.05616734
AFL     0.00166159
AGN     0.09038460
AIG     0.02888489
AIV     0.04016961
AIZ    -0.03898438
AJG     0.03285759
AKAM   -0.10153369
ALB     0.05947434
ALGN    0.10144223
ALK     0.14254027
ALL    -0.02343179
ALLE    0.04174912
           ...    
VRTX    0.12013115
VTR     0.04484575
VZ      0.00226219
WAT     0.12218932
WBA    -0.04948093
WDC     0.00527594
WEC    -0.00599034
WFC     0.01343147
WHR     0.14353282
WLTW   -0.00556610
WM     -0.00226603
WMB     0.10272181
WMT    -0.00068498
WRK     0.09888215
WU      0.04186420
WY      0.09448534
WYN 

Name: 2017-06-30 00:00:00, Length: 495, dtype: float64
-----3
ticker      A  AAL  AAP  AAPL  ABBV  ABC  ABT  ACN  ADBE  ADI ...   XL  XLNX  \
date                                                          ...              
2013-07-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2013-08-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2013-09-30  1    0    0     1     0    0    0    0     0    0 ...    0     0   
2013-10-31  0    1    0     0     0    0    0    0     1    0 ...    0     0   
2013-11-30  0    1    1     0     0    0    0    0     0    0 ...    0     0   
2013-12-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2014-01-31  0    0    1     0     1    0    0    0     0    0 ...    0     0   
2014-02-28  0    1    0     0     0    0    0    0     0    0 ...    0     0   
2014-03-31  0    0    0     0     0    0    0    0     1    0 ...    0     1   
2014-04-30  0    0    0     0     0    0    0    0     0  

ticker
A      -0.05388057
AAL    -0.06647111
AAP    -0.01828314
AAPL   -0.06772063
ABBV   -0.00000000
ABC    -0.07997603
ABT    -0.04389252
ACN    -0.05260536
ADBE   -0.04613431
ADI     0.02215021
ADM     0.01133043
ADP    -0.06445521
ADS    -0.02169826
ADSK   -0.12582635
AEE     0.00916291
AEP    -0.01532651
AES    -0.03349929
AET    -0.09473005
AFL    -0.02668102
AGN    -0.05377459
AIG     0.03747973
AIV     0.09918419
AIZ    -0.11260120
AJG     0.01936437
AKAM    0.00078234
ALB    -0.03736535
ALGN    0.04333713
ALK    -0.09813264
ALL    -0.02712574
ALLE           nan
           ...    
VRTX    0.02742233
VTR     0.13800842
VZ      0.01777736
WAT     0.01386902
WBA    -0.00455039
WDC    -0.07482580
WEC    -0.00121083
WFC    -0.03772331
WHR    -0.04951635
WLTW           nan
WM     -0.04798051
WMB     0.01381666
WMT    -0.05401922
WRK            nan
WU      0.02077843
WY      0.00941455
WYN    -0.08522091
WYNN   -0.02191910
XEC     0.10648565
XEL     0.02953801
XL     -0.04540422
XLNX 

-----1
2014-12-31 00:00:00
-----2
ticker
A      -0.06740128
AAL    -0.16010974
AAP    -0.00081622
AAPL   -0.10071836
ABBV   -0.08659108
ABC    -0.06719583
ABT    -0.02088612
ACN    -0.06224762
ADBE   -0.04952333
ADI    -0.09637224
ADM    -0.11869216
ADP    -0.04611975
ADS    -0.00885428
ADSK   -0.07465403
AEE    -0.01802272
AEP     0.00451706
AES     0.01431664
AET    -0.05574344
AFL    -0.00657398
AGN    -0.10868776
AIG    -0.02270098
AIV    -0.04715784
AIZ     0.00529102
AJG    -0.00522740
AKAM   -0.06903709
ALB    -0.01124181
ALGN   -0.07819907
ALK    -0.10567986
ALL    -0.05376893
ALLE   -0.01421382
           ...    
VRTX   -0.04547027
VTR    -0.04341679
VZ     -0.00674338
WAT    -0.04500098
WBA    -0.06612361
WDC    -0.04861104
WEC    -0.00279939
WFC    -0.03236218
WHR    -0.08303565
WLTW           nan
WM      0.00327802
WMB     0.07013875
WMT    -0.13781616
WRK            nan
WU     -0.09122810
WY     -0.04193183
WYN    -0.07493295
WYNN    0.04786072
XEC     0.07841967
XEL    -0

ticker
A      -0.09523073
AAL    -0.17421946
AAP    -0.04588907
AAPL   -0.08329065
ABBV   -0.09970324
ABC    -0.01587501
ABT    -0.11369172
ACN    -0.09776101
ADBE   -0.07541025
ADI    -0.06369612
ADM    -0.09673476
ADP    -0.08002355
ADS    -0.13802453
ADSK   -0.22341538
AEE    -0.03281272
AEP     0.00370012
AES    -0.12102784
AET    -0.05011964
AFL    -0.09228730
AGN    -0.12652130
AIG    -0.10419828
AIV    -0.05696331
AIZ    -0.03139662
AJG    -0.05765624
AKAM    0.12705701
ALB    -0.19359563
ALGN   -0.14260742
ALK     0.04111048
ALL    -0.06062462
ALLE   -0.12243555
           ...    
VRTX   -0.18049542
VTR     0.04263717
VZ     -0.08740288
WAT    -0.07800384
WBA    -0.01883474
WDC     0.17249593
WEC     0.01271939
WFC    -0.05290835
WHR    -0.08384870
WLTW           nan
WM     -0.07631393
WMB    -0.06792491
WMT     0.12466974
WRK    -0.04411362
WU     -0.04733668
WY     -0.08079183
WYN    -0.12348457
WYNN   -0.27522720
XEC    -0.14152531
XEL    -0.00619371
XL     -0.04732066
XLNX 

-----1
2017-03-31 00:00:00
-----2
ticker
A      -0.04648289
AAL    -0.04873915
AAP     0.04757013
AAPL   -0.12552425
ABBV   -0.01187489
ABC    -0.05135996
ABT    -0.07626031
ACN    -0.07305358
ADBE   -0.04281683
ADI    -0.09464624
ADM    -0.06665780
ADP    -0.01601134
ADS    -0.06424823
ADSK   -0.05919170
AEE    -0.03801464
AEP    -0.05362976
AES    -0.00696867
AET    -0.08210941
AFL    -0.03920285
AGN    -0.11480270
AIG     0.00530505
AIV    -0.06233585
AIZ    -0.02441744
AJG    -0.05634276
AKAM    0.09138147
ALB    -0.09143630
ALGN   -0.11398285
ALK    -0.04484890
ALL    -0.09294692
ALLE   -0.10018497
           ...    
VRTX   -0.05384041
VTR    -0.05335862
VZ     -0.01257113
WAT    -0.09000137
WBA    -0.05725361
WDC     0.03640016
WEC    -0.02944653
WFC    -0.03391829
WHR    -0.02093547
WLTW   -0.02603081
WM     -0.05350667
WMB     0.01748908
WMT    -0.06088900
WRK    -0.01410136
WU     -0.00305967
WY     -0.07351503
WYN    -0.05152094
WYNN    0.04841656
XEC     0.07216937
XEL    -0

## Projected Returns
It's now time to check if your trading signal has the potential to become profitable!

We'll start by computing the net returns this portfolio would return. For simplicity, we'll assume every stock gets an equal dollar amount of investment. This makes it easier to compute a portfolio's returns as the simple arithmetic average of the individual stock returns.

Implement the `portfolio_returns` function to compute the expected portfolio returns. Using `df_long` to indicate which stocks to long and `df_short` to indicate which stocks to short, calculate the returns using `lookahead_returns`. To help with calculation, we've provided you with `n_stocks` as the number of stocks we're investing in a single period.

In [14]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
np.multiply(x1, x2)
#print(x1)
#print(x2)

array([[  0.,   1.,   4.],
       [  0.,   4.,  10.],
       [  0.,   7.,  16.]])

In [15]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    # TODO: Implement Function
    print(df_long)
    print(lookahead_returns)
    print(df_short)
    print(n_stocks)
    print(top_bottom_n)
    rtn_long = np.multiply(df_long,lookahead_returns)
    rtn_short = np.multiply(df_short,lookahead_returns)
    rtn_long_short = rtn_long - rtn_short
    print(rtn_long_short)
    
    return rtn_long_short/n_stocks

project_tests.test_portfolio_returns(portfolio_returns)

            EJGN  TPE  SAR  ZWZM  BCB
2008-08-31     0    0    0     0    0
2008-09-30     0    0    0     0    0
2008-10-31     1    0    1     1    0
2008-11-30     0    1    0     1    1
                  EJGN         TPE         SAR        ZWZM         BCB
2008-08-31  3.13172138  0.72709204  5.76874778  1.77557845  0.04098317
2008-09-30 -3.78816218 -0.67583590 -4.95433863 -1.67093250 -0.24929051
2008-10-31  0.05579709  0.29199789  0.00697116  1.05956179  0.30686995
2008-11-30  1.25459098  6.87369275  2.58265839  6.92676837  0.84632677
            EJGN  TPE  SAR  ZWZM  BCB
2008-08-31     0    0    0     0    0
2008-09-30     0    0    0     0    0
2008-10-31     0    1    0     1    1
2008-11-30     1    1    1     0    0
3
50
                  EJGN         TPE         SAR       ZWZM         BCB
2008-08-31  0.00000000  0.00000000  0.00000000 0.00000000  0.00000000
2008-09-30  0.00000000  0.00000000  0.00000000 0.00000000  0.00000000
2008-10-31  0.05579709 -0.29199789  0.00697116 0.0

### View Data
Time to see how the portfolio did.

In [16]:
expected_portfolio_returns = portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)
project_helper.plot_returns(expected_portfolio_returns.T.sum(), 'Portfolio Returns')

ticker      A  AAL  AAP  AAPL  ABBV  ABC  ABT  ACN  ADBE  ADI ...   XL  XLNX  \
date                                                          ...              
2013-07-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2013-08-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2013-09-30  1    0    0     1     0    0    0    0     0    0 ...    0     0   
2013-10-31  0    1    0     0     0    0    0    0     1    0 ...    0     0   
2013-11-30  0    1    1     0     0    0    0    0     0    0 ...    0     0   
2013-12-31  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2014-01-31  0    0    1     0     1    0    0    0     0    0 ...    0     0   
2014-02-28  0    1    0     0     0    0    0    0     0    0 ...    0     0   
2014-03-31  0    0    0     0     0    0    0    0     1    0 ...    0     1   
2014-04-30  0    0    0     0     0    0    0    0     0    0 ...    0     0   
2014-05-31  0    0    0     1     0    0

ticker               A         AAL         AAP        AAPL        ABBV  \
date                                                                     
2013-07-31  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2013-08-31  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2013-09-30 -0.00960698 -0.14734639  0.00000000  0.09201927  0.00000000   
2013-10-31  0.00000000  0.06647111  0.00000000 -0.06772063  0.00000000   
2013-11-30  0.00000000  0.07267716  0.09197258  0.00000000  0.00000000   
2013-12-31  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2014-01-31  0.00000000  0.00000000  0.10373913  0.00000000  0.03355617   
2014-02-28  0.00000000 -0.00897599  0.00000000 -0.01975642  0.00000000   
2014-03-31  0.03182502  0.00000000  0.00000000  0.00000000  0.00000000   
2014-04-30  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2014-05-31  0.00000000  0.00000000  0.00000000  0.02728625  0.00000000   
2014-06-30  0.00000000 -0.09799068  0.

## Statistical Tests
### Annualized Rate of Return

In [17]:
expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))


Mean:                       0.003253
Standard Error:             0.002203
Annualized Rate of Return:  3.98%



The annualized rate of return allows you to compare the rate of return from this strategy to other quoted rates of return, which are usually quoted on an annual basis. 

### T-Test
Our null hypothesis ($H_0$) is that the actual mean return from the signal is zero. We'll perform a one-sample, one-sided t-test on the observed mean return, to see if we can reject $H_0$.

We'll need to first compute the t-statistic, and then find its corresponding p-value. The p-value will indicate the probability of observing a t-statistic equally or more extreme than the one we observed if the null hypothesis were true. A small p-value means that the chance of observing the t-statistic we observed under the null hypothesis is small, and thus casts doubt on the null hypothesis. It's good practice to set a desired level of significance or alpha ($\alpha$) _before_ computing the p-value, and then reject the null hypothesis if $p < \alpha$.

For this project, we'll use $\alpha = 0.05$, since it's a common value to use.

Implement the `analyze_alpha` function to perform a t-test on the sample of portfolio returns. We've imported the `scipy.stats` module for you to perform the t-test.

Note: [`scipy.stats.ttest_1samp`](https://docs.scipy.org/doc/scipy-1.0.0/reference/generated/scipy.stats.ttest_1samp.html) performs a two-sided test, so divide the p-value by 2 to get 1-sided p-value

In [27]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    # TODO: Implement Function
#    print(expected_portfolio_returns_by_date)
    std_value = expected_portfolio_returns_by_date.std()
    mean_value = expected_portfolio_returns_by_date.mean()
#    print(std_value)
#    print(mean_value)
    population_mean = 0
    t_value = (mean_value-population_mean)/std_value
    [tt,pp] = stats.ttest_1samp(expected_portfolio_returns_by_date,0.0)
#    print(['t-value1',t_value])
#    print(['t-value2',tt])
#    print(['p-value',pp])
    
    return tt,pp/2

project_tests.test_analyze_alpha(analyze_alpha)

Tests Passed


### View Data
Let's see what values we get with our portfolio. After you run this, make sure to answer the question below.

In [28]:
t_value, p_value = analyze_alpha(expected_portfolio_returns_by_date)
print("""
Alpha analysis:
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(t_value, p_value))


Alpha analysis:
 t-value:        1.476
 p-value:        0.073359



### Question: What p-value did you observe? And what does that indicate about your signal?

*#TODO: Put Answer In this Cell*
The p-value is "0.073 > 0.05", which means that the profitability does not pass the null-hypothesis. In the other words, only using the single momentum alpha factor is insufficient to achieve our goal.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.